In [0]:
%pip install -r ../../requirements.txt
%restart_python

In [0]:
import yfinance as yf
from datetime import datetime, date
import os
import time
from pyspark.sql.functions import lit
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException
from pyspark.sql import functions as F
from pyspark.sql.types import DateType, TimestampType
import yaml


try:
    # dbutils.widgets.get() jest standardową metodą odczytu parametrów Joba
    ENV = dbutils.widgets.get("env_name") 
    print(f"Środowisko skonfigurowane przez Job (ENV): {ENV}")
except Exception:
    # Obsługa błędu - gdy uruchamiasz notebook interaktywnie lub ręcznie (spoza Joba),
    # dbutils.widgets.get() może zgłosić błąd, bo widgety nie są zainicjowane.
    # W takim przypadku ustawiamy wartość domyślną (np. 'TEST')
    ENV = 'TEST' 
    print(f"Uruchomienie interaktywne/ręczne. Użycie domyślnego ENV: {ENV}")


try:
    # W Databricks, ścieżka do pliku YAML może wymagać dostosowania,
    # jeśli nie jest on w tym samym katalogu co notatnik.
    with open('../../config/config.yaml', 'r') as file:
        full_config = yaml.safe_load(file)
except FileNotFoundError:
    print("BŁĄD: Plik 'config.yaml' nie został znaleziony! Sprawdź ścieżkę.")
    raise

CFG = full_config.get(ENV)
if not CFG:
    raise ValueError(f"Nie znaleziono konfiguracji dla środowiska: {ENV} w pliku YAML.")

catalog_name = CFG['catalog_name']
schema_name = CFG['schema_name']
volume_name = CFG['volume_name']
sql_table = CFG['sql_table_name']
base_output_directory = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/yfinance_delta_lake_bronze/"

In [0]:
# --- KROK 1: Tworzenie df_tickets z tabeli Databricks SQL ---
try:
    sql_query = f"SELECT Ticket, company_name FROM {sql_table}"
    spark_df_tickets = spark.sql(sql_query)
    df_tickets_list = spark_df_tickets.collect()
    company_info_dict = {row['Ticket']: row['company_name'] for row in df_tickets_list}
    gpw_ticket_list = list(company_info_dict.keys())
    print("Pomyślnie utworzono Spark DataFrame z tabeli SQL.")
    print("Akcje do przetworzenia:", gpw_ticket_list)
except Exception as e:
    print(f"Błąd odczytu z tabeli SQL: {e}.")
    gpw_ticket_list = []
    company_info_dict = {}


In [0]:
# Utworzenie katalogu bazowego, jeśli nie istnieje
try:
    dbutils.fs.mkdirs(base_output_directory)
    print(f"Pomyślnie utworzono katalog: {base_output_directory}")
except Exception as e:
    print(f"Katalog już istnieje lub wystąpił błąd podczas jego tworzenia: {e}")

# Ustawienia zakresu dat dla yfinance
end_date = date.today()

if not gpw_ticket_list:
    print("Brak danych firm do przetworzenia. Używanie danych zastępczych dla demonstracji.")




In [0]:
from pyspark.sql.functions import to_date
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql import types as T
# --- DODATKOWY IMPORT DLA DELTA LAKE ---
from delta.tables import DeltaTable
from pyspark.sql.functions import col

# Helper function to check if a path is a Delta Table
def is_delta_table(path):
    """Sprawdza, czy podana ścieżka zawiera folder _delta_log (czyli jest Delta Table)."""
    try:
        # dbutils.fs.ls zgłosi błąd, jeśli ścieżka nie istnieje.
        dbutils.fs.ls(os.path.join(path, "_delta_log")) 
        return True
    except Exception:
        return False

print("\nRozpoczynanie pobierania danych z yfinance i zapisywanie do Delta Lake...")

for t in gpw_ticket_list:
    yfinance_ticker = f"{t}.WA"
    output_path = os.path.join(base_output_directory, f"ticker={t}")
    print(f"Przetwarzanie akcji: {yfinance_ticker}")
    is_existing_delta = is_delta_table(output_path)

    # --- LOGIKA POBIERANIA DANYCH ---
    start_date_for_update = None
    if is_existing_delta:
        try:
            # Odczytujemy maksymalną datę z istniejącej tabeli Delta
            delta_table = spark.read.format("delta").load(output_path)
            # Konwersja kolumny na typ DateType (jeśli nie była DateType)
            delta_table = delta_table.withColumn("Date", col("Date").cast("date"))
            last_download_date = delta_table.selectExpr("max(Date)").collect()[0][0]

            if last_download_date and last_download_date < date.today():
                # Ustawiamy datę początkową na dzień po ostatnio pobranym
                start_date_for_update = (pd.to_datetime(last_download_date) + pd.Timedelta(days=1)).strftime('%Y-%m-%d')
                print(f"Dane dla '{yfinance_ticker}' wymagają aktualizacji od daty: {start_date_for_update}.")
            elif last_download_date:
                 print(f"Dane dla '{yfinance_ticker}' są aktualne na dzień {last_download_date}. Pomijanie.")
                 time.sleep(1) # Unikaj przeciążania API
                 continue
            else:
                 # Jeśli last_download_date jest None (tabela pusta, ale istnieje)
                 print(f"Ostrzeżenie: Istniejąca tabela Delta pusta dla '{yfinance_ticker}'. Pełne ponowne pobranie.")
        except Exception as e:
            # Błąd odczytu Delta Table, traktujemy jak konieczność pełnego pobierania
            print(f"Wystąpił błąd odczytu Delta Table dla '{yfinance_ticker}': {e}. Pełne ponowne pobranie.")
            is_existing_delta = False
            
    # Ustawienia daty początkowej dla yfinance
    if not is_existing_delta or (is_existing_delta and not last_download_date):
        # Pełne pobieranie od maksymalnej dostępnej daty
        data = yf.download(yfinance_ticker, period='max', end=end_date)
        print(f"Brak istniejących danych Delta dla '{yfinance_ticker}'. Rozpoczęcie pełnego pobierania.")
    elif start_date_for_update:
        # Pobieranie tylko nowych danych
        data = yf.download(yfinance_ticker, start=start_date_for_update, end=end_date)
    else:
        # Dane aktualne (powinno być obsłużone w bloku powyżej, ale zabezpieczenie)
        time.sleep(1)
        continue
    # --- LOGIKA ZAPISU DO DELTA LAKE ---
    if not data.empty:
        # Przygotowanie danych do zapisu Spark DataFrame
        df_new_data = data.reset_index()
        # Obsługa MultiIndex po pobraniu (jeśli wystąpi)
        if isinstance(df_new_data.columns, pd.MultiIndex):
            df_new_data.columns = df_new_data.columns.map(lambda x: x[0])
        # Konwersja do Spark DataFrame
        spark_new_data_df = spark.createDataFrame(df_new_data)

        # Czyszczenie i dodawanie kolumn
        spark_new_data_df = spark_new_data_df.withColumn("Date", col("Date").cast("date")) \
                                              .withColumn("Ticket", lit(t)) \
                                              .withColumn("company_name", lit(company_info_dict.get(t, 'N/A')))
        # Upewnienie się, że DF ma wymagane kolumny
        cols_to_select = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Ticket', 'company_name']
        spark_new_data_df = spark_new_data_df.select([F.col(c) for c in cols_to_select if c in spark_new_data_df.columns])

        # A) Jeśli tabela NIE ISTNIEJE, tworzymy nową jako Delta Lake
        if not is_existing_delta:
            try:
                # Tworzymy pierwszą wersję Delta Table (partycjonowaną)
                spark_new_data_df.write.format("delta").mode("overwrite").partitionBy("Ticket").save(output_path)
                print(f"Pobrano i zapisano **pełną nową** Delta Table dla '{yfinance_ticker}'.")
            except Exception as e:
                print(f"BŁĄD ZAPISU PEŁNEJ DELTA TABLE dla '{yfinance_ticker}': {e}")
        # B) Jeśli tabela ISTNIEJE, używamy MERGE
        else:
            try:
                # Używamy MERGE INTO do transakcyjnej aktualizacji/wstawiania
                deltaTable = DeltaTable.forPath(spark, output_path)
                
                deltaTable.alias("target") \
                    .merge(
                        source=spark_new_data_df.alias("source"),
                        # Łączymy na podstawie 'Date' i 'Ticket' (kolumny partycji)
                        condition="target.Date = source.Date AND target.Ticket = source.Ticket"
                    ) \
                    .whenMatchedUpdateAll() \
                    .whenNotMatchedInsertAll() \
                    .execute()
                
                print(f"Zaktualizowano dane dla '{yfinance_ticker}' za pomocą **MERGE INTO** Delta Lake. Dodano/zaktualizowano {spark_new_data_df.count()} wierszy.")
            except Exception as e:
                print(f"BŁĄD MERGE INTO DELTA LAKE dla '{yfinance_ticker}': {e}. Pełne ponowne pobranie mogło by pomóc.")
                
    else:
        print(f"Brak nowych danych do pobrania dla '{yfinance_ticker}'.")
        
    time.sleep(1) # Unikaj przeciążania API
    
print("\n--- ZAKOŃCZONO PROCES AKTUALIZACJI/WSTAWIANIA DO DELTA LAKE ---")
